#Loading the required libraries

In [31]:
! pip install selenium
!apt-get update
!apt install chromium-chromedriver
! pip install pattern

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [32]:
import selenium
import time
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import sys

In [33]:
# Creating Driver Instance
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# Loading the movies dataset

In [34]:
def load_data():
  import pandas as pd
  global movie
  global link
  movie = pd.read_csv('/content/drive/MyDrive/MovieLens Dataset(1)/movie.csv')
  link = pd.read_csv('/content/drive/MyDrive/MovieLens Dataset(1)/link.csv')
  global movie_links
  movie_links = pd.concat([movie,link],axis=1,join='inner')
  movie_links.drop('tmdbId',axis=1,inplace=True)
  df = movie_links['title'].str.split('(',expand=True)
  df.rename({0:'title',1:'year'},axis=1,inplace=True)
  df = df[['title','year']]
  df['year'] = df['year'].str.replace(')','')
  movie_links['year'] = df['year']
  movie_links['title'] = df['title']
  movie_links.index = movie_links['title']




#Scraping the Data

In [35]:
def extend_page(wd):
  count = 20
  from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
  prev_height = wd.execute_script('return document.body.scrollHeight')


  while(count>0):
    try:

      wd.find_element_by_id("load-more-trigger").click()
      print(count)

    except (ElementNotInteractableException,NoSuchElementException):
      page = wd.page_source
      return page
    time.sleep(1)

    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    new_height = wd.execute_script('return document.body.scrollHeight')

    prev_height = new_height
    count-=1
  page = wd.page_source
  return page


In [36]:
def scrape_reviews(page):
  from bs4 import BeautifulSoup as bsoup
  import requests
  soup = bsoup(page,'lxml')
  revs = soup.find_all('div',{'class':'text show-more__control'})
  revs2 = soup.find_all('div',{'class':'text show-more__control clickable'})
  rating = soup.find_all('div',{'class':'ipl-ratings-bar'})
  ratings = []
  global k
  k = 0
  reviews = []
  for j,i in tqdm(enumerate(revs)):
    reviews.append(i.text)
    k+=1

  for j,i in tqdm(enumerate(revs2)):
    reviews.append(i.text)
    k+=1

  for j,i in tqdm(enumerate(rating)):
    ratings.append(i.text.split('/')[0][-1])

  ratings = [int(i) for i in ratings]
 
  print(k,'reviews scraped')

  return reviews,ratings

# Preprocessing

In [37]:
def preprocess(reviews):
    import re
    import gensim
    from gensim.parsing.preprocessing import remove_stopwords
    from gensim.models.phrases import Phrases,Phraser
    from gensim.utils import lemmatize
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    cleaned_reviews = []
    for rev in reviews:
        text = re.sub('[^a-zA-Z]',' ',rev)
        text = text.lower()
        text = re.sub('(\\d|\\W)+',' ',text)
        text = remove_stopwords(text)
        cleaned_reviews.append(text.split())

    lemmatized_reviews = []


    for review in cleaned_reviews:
        lemmatized_review = []
        for word in review:
            lemmatized_review.append(lemmatizer.lemmatize(word))
        lemmatized_reviews.append(lemmatized_review)
        

    phrases = Phrases(lemmatized_reviews)
    bigram = Phraser(phrases)

    final_phrases = bigram[lemmatized_reviews]
    
    return final_phrases


# Data Labelling

In [38]:
def labelling(reviews):
  print('labelling')
  import nltk
  import pandas as pd
  nltk.download('vader_lexicon')

  from nltk.sentiment.vader import SentimentIntensityAnalyzer

  sid = SentimentIntensityAnalyzer()

  labels = []

  for review in reviews:

    score = sid.polarity_scores(review)

    if (score['compound'] >= 0.05 ):
      labels.append(1)

    elif (score['compound'] <= -0.05):
      labels.append(2)

    else:
      labels.append(0)


  df = pd.DataFrame()
  print('reviews:',len(reviews))
  print('labels:',len(labels))


  df['reviews'] = reviews
  df['label'] = labels

  return df


# Embedding

In [39]:
def gensim_embedding(all_phrases1):
  print('embedding')
  print('Shape of phrases',all_phrases1.shape)
  import gensim
  import numpy as np
  from gensim.models import Word2Vec as wvec
  from sklearn.decomposition import PCA

  all_phrases1['cleaned_reviews2'] = all_phrases1['cleaned_reviews']

  cbow = wvec(min_count=10,window=5,size=all_phrases1.shape[0],alpha=0.01,min_alpha=0.001,sample=6e-5,negative = 10)


  dictionary = gensim.corpora.Dictionary(all_phrases1['cleaned_reviews2'])

  model = wvec(min_count=10,window=5,size=all_phrases1.shape[0],alpha=0.01,min_alpha=0.001,sample=6e-5,negative = 10)

  model.build_vocab(sentences=all_phrases1['cleaned_reviews2'])

  model.train(all_phrases1,total_examples=cbow.corpus_count,epochs=200,)

  vectors = model.wv.vectors

  print(vectors.shape)

  pca = PCA(n_components=100)

  v2 = vectors.T
  # v2 = vectors


  print(v2.shape)


  v2 = pca.fit_transform(v2) 

  return v2


# Data Splitting and Training

In [40]:
def data_prep(phrases,vectors):
  labels = phrases['label']
  import keras
  from keras.utils import to_categorical
  import numpy as np

  labels = np.array(labels)

  labels2 = to_categorical(labels,3)
  vectors = vectors.reshape((vectors.shape[0],vectors.shape[1],1))
  model = keras.models.load_model('/content/drive/MyDrive/NLP Project/lstm_classifier_4.0.h5')

  pred = predict(model,vectors,labels2)

  return pred

In [41]:
def predict(lstm,x_test,y_test):

  from sklearn.metrics import accuracy_score,confusion_matrix
  from keras.utils import to_categorical

  print('evaluating:')

  lstm.evaluate(x_test,y_test)

  revs = []

  pred = lstm.predict(x_test)


  print('shape of pred:',pred.shape)


  print('0th index:',(pred[0]))

  revs = []


  for i in range(len(pred)):
    max_ind = np.argmax(pred[i],axis=0)
    revs.append(max_ind)
    # print('revs length:',len(revs))




  revs = np.array(revs)

  revs = to_categorical(revs,3)


  print('Accuracy on testing data:',accuracy_score(revs,y_test))

  print('\n')

  print(confusion_matrix(y_test.argmax(axis=1),revs.argmax(axis=1)))

  revs = revs.argmax(axis=1)

  return revs





# Plotting

In [42]:
def plot(pred,phrases,ratings,remove_words):

  import numpy as np
  import pandas as pd
  from os import path
  from PIL import Image
  from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
  import matplotlib.pyplot as plt


  neg = len(pred[pred==2])
  pos = len(pred[pred==1])
  neu = len(pred[pred==0])
  groups = [pos,neg,neu]

  labels = ['Positive','Negative','Neutral']
  plt.rcParams['figure.figsize']=(17,10)
  fig,axs = plt.subplots(2,2)

  axs[0,0].hist(ratings,bins=np.arange(11)-0.5,rwidth=0.5)
  axs[0,0].set_xticks(range(10),)
  axs[0,0].set_xticklabels([1,2,3,4,5,6,7,8,9,10])

  axs[0,1].pie(groups,labels=labels,autopct='%.2f')

  gensim_stopwords = STOPWORDS.union(set(remove_words))


  all_text = [text for i in phrases['reviews'] for text in i]
  all_text = ''.join(phrases['reviews'][:])
  # all_text

  filtered_text = [word for word in all_text if word not in gensim_stopwords]
  wc = WordCloud().generate(filtered_text)

  axs[1,0].imshow(wc)
  axs[1,0].set_xticks([])
  axs[1,0].set_yticks([])
  axs[1,0].set_title('High frequency words present in the scraped reviews')





  plt.show()

In [43]:
def html_plot(pred,phrases,ratings,remove_words):

  !pip install mpld3
  import nltk
  nltk.download('stopwords')
  import numpy as np
  import pandas as pd
  from os import path
  from PIL import Image
  import gensim
  from gensim.parsing.preprocessing import STOPWORDS
  from nltk.corpus import stopwords
  from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
  import matplotlib.pyplot as plt
  import mpld3


  neg = len(pred[pred==2])
  pos = len(pred[pred==1])
  neu = len(pred[pred==0])
  groups = [pos,neg,neu]

  labels = ['Positive','Negative','Neutral']

  # fig = plt.figure()
  plt.rcParams['figure.figsize']=(17,10)
  fig,axs = plt.subplots(2,2)


  axs[0,0].hist(ratings,bins=np.arange(11)-0.5,rwidth=0.5)
  axs[0,0].set_xticks(range(10),)
  axs[0,0].set_xticklabels([1,2,3,4,5,6,7,8,9,10])
  axs[0,0].set_xlabel('Ratings')
  axs[0,0].set_ylabel('Number of Users')
  axs[0,0].set_title('Frequency Distribution of Ratings',fontsize=15)


  axs[0,1].pie(groups,labels=labels,autopct='%.2f')
  axs[0,1].set_title('Percentage of positive, negative and neutral reviews',fontsize=15)
  axs[0,1].xaxis.set_visible(False)
  axs[0,1].yaxis.set_visible(False)
  nltk_stopwords = stopwords.words("english")

  nltk_stopwords.extend(remove_words)


  all_text = [text for i in phrases['reviews'] for text in i]
  all_text = ' '.join(phrases['reviews'][:])
  print('removing stopwords...')
  gensim_stopwords = STOPWORDS.union(set(remove_words))
  # all_text
  all_text_list = all_text.split()
  filtered_text_list = [word for word in all_text_list if word not in nltk_stopwords]

  # print('diff between all_text and filtered_text_list:',len(all_text_list)==len(filtered_text_list))

  filtered_text = ' '.join(filtered_text_list)

  # print('filtered_text:',filtered_text)
  # print('all_text:',all_text_list[:10])


  wc = WordCloud().generate(all_text)
  wc = WordCloud().generate(filtered_text)



  axs[1,0].imshow(wc)
  axs[1,0].set_xticks([])
  axs[1,0].set_yticks([])
  axs[1,0].set_title('High frequency words present in the scraped reviews',fontsize=15)


  # axs[1,1].set_xticks([])
  # axs[1,1].set_yticks([])
  axs[1,1].xaxis.set_visible(False)
  axs[1,1].yaxis.set_visible(False)


  plt.savefig("plot.png")

  html_plot2 = mpld3.fig_to_html(fig)
  c = '''<!doctype html>
  <html>
  <head>
  </head>
  <body style="background-image: linear-gradient(blue,darkblue); height: 700px;">

  <img src = "{{url_for('static',filename='/content/plot.png')}}"
  width='500'
  height='500'
  style="padding-top:150px;padding-left:500px"
  >

  </body>
  </html>'''


  html_file2 = open("/content/templates/plot.html","w") # Saving the html file
  html_file2.write(html_plot2) # Writing the file
  html_file2.close() # Closing the file

  return html_file2
  # return 0


In [44]:
def scrape_stopwords(wd,id):
  wd.get("https://www.imdb.com/title/{}/fullcredits?ref_=tt_ql_1".format(id))

  count = 10
  rev1 = []
  prev_height = wd.execute_script('return document.body.scrollHeight')

  while(count>0):
    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(1)
    new_height = wd.execute_script('return document.body.scrollHeight')
    count-=1
  page = wd.page_source

  from bs4 import BeautifulSoup as bsoup
  from tqdm import tqdm
  import requests
  soup = bsoup(page,'lxml')
  revs = soup.find_all('a')
  remove_words = []
  for j,i in tqdm(enumerate(revs)):
    remove_words.append(i.text)

  remove_words2 = []
  for j in remove_words:

    if(j=='\n'):
      continue

    if(' ' in j):

      remove_words2.extend(j.split())

    else:
      remove_words2.append(j)





  print(remove_words2)

  return remove_words2



# Driver function

In [60]:
def get_movie(selection):
  load_data()
  movie = selection
  movie = movie.lower()
  info = movie.title()
  info = info+' '
  print(info)

  if (info not in movie_links['title']):

    if (info[:3]=='The'):
      l1 = info.split()

      del l1[0]

      info = ' '.join(l1)
      info+=', The '
      print(info)
      if (info not in movie_links['title']):
        print('Movie not recognized')
        return 0

    else:
        print('Movie not recognized')
        return 0



  id = str(movie_links.imdbId.loc[info])
  id = 'tt0'+id
  wd = webdriver.Chrome('chromedriver',options=options)

  wd.get('https://www.imdb.com/title/{}/reviews?ref_=tt_urv'.format(id))

  print('Extending the page...')
  
  page = extend_page(wd)

  reviews,ratings = scrape_reviews(page)
  

  


  print(len(reviews))

  if (len(reviews)>=100):
    print('Labelling the reviews...')
    phrases = labelling(reviews) 

    print('preprocessing the reviews...')
    phrases['cleaned_reviews'] = preprocess(phrases['reviews'])

    print('Embedding...')

    vectors = gensim_embedding(phrases)


    print('Data collection and preprocessing complete.')


    pred = data_prep(phrases,vectors)

    remove_words = scrape_stopwords(wd,id)

    remove_words.append(selection)
    remove_words.append(movie)
    remove_words.append(movie.title())

    remove_words = list(set(remove_words))



    plot_file = html_plot(pred,phrases,ratings,remove_words)

    return plot_file

  else:
    # mnr = '''
    # <!DOCTYPE html>
    # <html>
    #     <head style="text-align: center;">Movie Not Recognized</head>
    #     <body>
    #         <h1 style="text-align: center;">Movie Not Recognized</h1>
    #     </body>
    # </html>'''

    # html_file3 = open("content/templates/mnr.html","w")
    # html_file3.write(mnr)
    # html_file3.close()
    # return html_file3
    str2 = "Not enough reviews"
    return str2
  


# Deployment

In [46]:
!pip install flask-ngrok

In [47]:
from flask_ngrok import run_with_ngrok
from flask import Flask

In [48]:
def load_mymodel():
    import keras
    global mymodel
    mymodel = keras.models.load_model('/content/drive/MyDrive/NLP Project/imdb_classifier_4.0.h5')
    return mymodel


In [62]:
home = '''<!doctype html>
<html lang="en">

<title>IMDb Reviewer</title>

<head>
    <title>Imdb Reviewer</title>
</head>

<body style="background-image: linear-gradient(blue,darkblue); height: 750px;">
    <h1 style="text-align:center;color:black;padding-top: 200px;" >IMDb Reviewer</h1>
    <div class="jumbotron text-center"></div>
    <form style="text-align:center;" action="{{url_for('load_movie')}}" method="post">
    <label for="movie" style="font-size:25px;">Enter the movie whose reviews you would like to see:</label>
    <input align="center" type="text" style="background-color:black; font-size: 20px;color:white;" id="movie" name="film" placeholder="Search">
    <button type="submit" style="background-color:darkblue;color:white;font-size:20px;">Enter</button>
  </form>
 </div
</body>

</html>'''


ner = '''<!DOCTYPE html>
<html>
    <head>
        <title>Not enough reviews to scrape for this movie</title>
    </head>

    <body style= "background-image: linear-gradient(blue,darkblue); height: 750px;">
        <h1 style="text-align: center;color:black;padding-top:200px;">There are not enough reviews given for this movie. Please try again later.</h1>
    </body>
</html>'''


mnr = '''<!DOCTYPE html>
<html>
    <head style="text-align: center;"></head>
    <body style= "background-image: linear-gradient(blue,darkblue); height: 750px;">
        <h1 style="text-align: center;">Movie Not Recognized</h1>
    </body>
</html>'''

In [63]:
!mkdir '/content/templates'

index_file = open('/content/templates/index.html',"w")
index_file.write(home)
index_file.close()

ner_file = open('/content/templates/ner.html','w')
ner_file.write(ner)
ner_file.close()


mnr_file = open('/content/templates/mnr.html','w')
mnr_file.write(mnr)
mnr_file.close()

In [64]:
from flask import Flask,request,render_template

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
  return render_template("index.html")

@app.route('/',methods=["GET","POST"])

def load_movie():
  if request.method=="POST":
    movie = request.form.get("film")

    plot_file = get_movie(movie)

    print(type(plot_file))

    if(type(plot_file)!=str and plot_file!=0):
      return render_template("plot.html")

    if(plot_file==0):
      return render_template("mnr.html")
      pass

    if(type(plot_file)==str):
      return render_template("ner.html")

if __name__=='__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6bad1b0eb076.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/May/2021 11:50:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2021 11:50:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/May/2021 11:50:34] "POST / HTTP/1.1" 200 -


Khgfd 
Movie not recognized
<class 'int'>


127.0.0.1 - - [08/May/2021 11:50:43] "GET / HTTP/1.1" 200 -


Blackthorn 
Extending the page...
20
19


39it [00:00, 23156.55it/s]
19it [00:00, 15583.06it/s]
49it [00:00, 43349.69it/s]
127.0.0.1 - - [08/May/2021 11:50:54] "POST / HTTP/1.1" 200 -


58 reviews scraped
58
<class 'str'>


127.0.0.1 - - [08/May/2021 11:51:00] "GET / HTTP/1.1" 200 -
